In [1]:
import pandas as pd
import requests
import numpy as np
import json
from bs4 import BeautifulSoup as bs
import random
import time
import json
import pickle

In [36]:
with open("listas_usuarios_10.pickle", "rb") as archivo:
    anime_listas = pickle.load(archivo)

In [37]:
with open("usuarios_10.json", "r", encoding="utf-8") as archivo:
    usuarios = json.load(archivo)

In [38]:
{int(x) for x in usuarios}.difference(set(anime_listas))

{7786273,
 7788980,
 7790304,
 7796821,
 7796844,
 7799160,
 7802116,
 7807118,
 7808307,
 7808377,
 7808881,
 7812214,
 7816939,
 7820687,
 7820772,
 7821714,
 7824508,
 7825780,
 7828880,
 7831699,
 7835115,
 7838441,
 7839077,
 7840941,
 7843162,
 7848973,
 7849616,
 7849797,
 7850943,
 7851294,
 7852069,
 7857010,
 7861437,
 7865790,
 7869208,
 7870717,
 7871501,
 7878371,
 7878452,
 7880387,
 7885789,
 7886168,
 7888494,
 7890129,
 7895655,
 7895904,
 7901038,
 7902985,
 7906471,
 7908687,
 7909396,
 7915764,
 7917728,
 7919739,
 7922615,
 7925117,
 7925667,
 7931822,
 7933593,
 7936307,
 7942619,
 7944727,
 7949173,
 7949263,
 7950118,
 7951159,
 7952485,
 7957917,
 7960117,
 7962427,
 7963300,
 7967445,
 7967630,
 7967697,
 7968667,
 7968682,
 7971139,
 7971236,
 7975212,
 7984005,
 7984370,
 7985151,
 7985470,
 7986615,
 7987249,
 7988965,
 7990160,
 7990630,
 8000845,
 8003416,
 8014028,
 8014104,
 8014624,
 8016624,
 8023426,
 8024535,
 8024946,
 8027349,
 8027870,
 8029702,


In [39]:
anime_listas

{7844116: {'nombre': 'ark616',
  1: [10, 2],
  205: [10, 2],
  11061: [10, 2],
  14719: [10, 2],
  20899: [10, 2],
  26055: [10, 2],
  31933: [10, 2],
  32182: [10, 2],
  34537: [10, 2],
  164: [9, 2],
  4081: [9, 2],
  5300: [9, 2],
  10379: [9, 2],
  28891: [9, 2],
  30276: [9, 2],
  32935: [9, 2],
  34021: [9, 2],
  36028: [9, 2],
  36098: [9, 2],
  37510: [9, 2],
  37989: [9, 2],
  2890: [8, 2],
  5114: [8, 2],
  7785: [8, 2],
  16498: [8, 2],
  20583: [8, 2],
  28223: [8, 2],
  30240: [8, 2],
  31964: [8, 2],
  32281: [8, 2],
  33486: [8, 2],
  35120: [8, 2],
  36456: [8, 2],
  36896: [8, 2],
  9989: [7, 2],
  10588: [7, 2],
  13601: [7, 2],
  14407: [7, 2],
  21473: [7, 2],
  22319: [7, 2],
  24543: [7, 2],
  28171: [7, 2],
  28805: [7, 2],
  29803: [7, 2],
  30831: [7, 2],
  31043: [7, 2],
  31149: [7, 2],
  31645: [7, 2],
  32282: [7, 2],
  32937: [7, 2],
  33489: [7, 2],
  34577: [7, 2],
  34933: [7, 2],
  35073: [7, 2],
  35788: [7, 2],
  36949: [7, 2],
  23755: [6, 2],
  278

In [40]:
usuarios[str(7786273)]

'josuevargas'

In [57]:
head = {
    "accept": "application/json",
    "X-MAL-CLIENT-ID": "dba80244ba88ffa1c4914806e9ccb3e4"
}

### Parametros para obtener la animelist del usuario.
parametros = {
    "fields": "list_status",
    "sort": "list_score",
    "limit": "1000"
}
user_animelist = requests.get(
    url="https://api.myanimelist.net/v2/users/"+usuarios[str(8444305)]+"/animelist",
    headers=head,
    params=parametros
)

In [58]:
print(user_animelist.text)
print(user_animelist.status_code)

{"data":[],"paging":{}}
200


In [59]:
usuarios = 0
for i in range(1, 16):
    with open("listas_usuarios_10.pickle", "rb") as archivo:
        usuarios += len(pickle.load(archivo))
usuarios

9120